# Expedia Personalised Hotel Searches
VU Data Mining Techniques 2024 | Assignment 2 | Group 17

## Setup & Data Loading

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train_path = 'data/training_set_VU_DM.csv'
test_path = 'data/test_set_VU_DM.csv'

In [3]:
df = pd.read_csv(train_path)
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


## EDA
###### ToDo Jaime

In [ ]:
competitive_metrics = ['compX_rate', 'compX_inv', "compX_rate_percent_diff"]
full_competitive_metrics = []
for metric in competitive_metrics:
    metric_list = []
    for x in range(1,9):
        metric_list.append(metric.replace("X", str(x)))
    full_competitive_metrics.append(metric_list)

In [ ]:
len(df)

In [ ]:
len(df[full_competitive_metrics[2]].dropna(axis = 'index', how = 'all'))

too much nan values

In [ ]:
len(df[full_competitive_metrics[1]].dropna(axis = 'index', how = 'all'))

feature not relevant enough.

In [ ]:
df['combined_comp_rate'] = df[full_competitive_metrics[0]].iloc[:, :].sum(axis=1)

In [ ]:
comp_rate_nan_index = df.index.difference(df[full_competitive_metrics[0]].dropna(axis = 'index', how = 'all').index)
df.loc[comp_rate_nan_index, 'combined_comp_rate'] = np.nan

In [ ]:
print(len(pd.unique(df['srch_id'])))
print(df['click_bool'].sum())
print(df['booking_bool'].sum())

On average more clicks than searches. On average less bookings than searches. Makes sense

In [ ]:
def plot_vars_dist(df):
    variables = [var for var in df if var not in ['srch_id','date_time']]  # ignoring these as they are only 1 value indicating occurence of a call/sms
    fig, axes = plt.subplots(nrows=len(variables), ncols=1, figsize=(10, 6 * len(variables)))

    for ax, var in zip(axes.flatten(), variables):
        var_data = df[var]
        mean = var_data.mean()
        std = var_data.std()
        # Adjust bins for better visualization based on data range and characteristics
        bins = min(30, int(var_data.nunique()))  # Use a minimum of 30 bins or less if fewer unique values

        ax.hist(var_data, bins=bins, alpha=0.75, color='blue', edgecolor='black', label=f'{var} Scores')
        ax.set_title(f'Distribution of {var} values')
        ax.set_xlabel(f'{var.capitalize()} Score')
        ax.set_ylabel('Frequency')
        ax.grid(axis='y', alpha=0.75)
        legend_label = f"Mean: {mean:.2f}, Std: {std:.2f}"
        ax.legend([f"{var.capitalize()} Scores\n{legend_label}"], loc='upper right', title='Statistics', frameon=True)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_vars_dist(df)

In [ ]:
df_corr = df.drop(['srch_id','date_time'],axis = 1)

In [ ]:
df_corr.corr()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

## Data Preprocessing
###### ToDo Ryan

### Datatype Conversion & Grouping Features

In [4]:
df['date_time'] = pd.to_datetime(df['date_time'])

boolean_features = ['prop_brand_bool', 'promotion_flag', 'srch_saturday_night_bool', 'random_bool', 'click_bool', 'booking_bool']
for col in boolean_features:
    df[col] = df[col].astype('bool')
    
categorical_features = ['site_id', 'visitor_location_country_id', 'prop_country_id']  # 'srch_id', 'prop_id' and 'srch_destination_id' are not included because they are real identifiers rather than categorical features
for col in categorical_features:
    df[col] = df[col].astype('category')
    
numerical_features = [col for col in df.columns if col not in boolean_features + categorical_features + ['date_time', 'srch_id', 'prop_id', 'srch_destination_id']]
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   srch_id                      int64         
 1   date_time                    datetime64[ns]
 2   site_id                      category      
 3   visitor_location_country_id  category      
 4   visitor_hist_starrating      float64       
 5   visitor_hist_adr_usd         float64       
 6   prop_country_id              category      
 7   prop_id                      int64         
 8   prop_starrating              int64         
 9   prop_review_score            float64       
 10  prop_brand_bool              bool          
 11  prop_location_score1         float64       
 12  prop_location_score2         float64       
 13  prop_log_historical_price    float64       
 14  position                     int64         
 15  price_usd                    float64       
 16  

### Removing Outliers for Numerical Features

!ToDo: Come back to this once we have something like boxplots to see what our data looks like

In [5]:
# for feature in numerical_features:
#     print(feature)
#     # Calculating IQR
#     Q1 = df[feature].quantile(0.25)
#     Q3 = df[feature].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR  # increase to 3 if we want to be less strict
#     upper_bound = Q3 + 1.5 * IQR
    
#     if df[(df[feature] < lower_bound) | (df[feature] > upper_bound)].any(axis=None):
#         print(f"{feature} has outliers")
#         print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
#         print(f"Number of outliers: {df[(df[feature] < lower_bound) | (df[feature] > upper_bound)].shape[0]}")
#         print(f"Percentage of outliers: {df[(df[feature] < lower_bound) | (df[feature] > upper_bound)].shape[0] / df.shape[0] * 100:.2f}%")
#         print("\n")

In [6]:
num_feats_with_outliers = ['price_usd', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff', 'srch_booking_window', 'orig_destination_distance']
outliers = pd.Series(data=False, index=df.index)

for feature in ['price_usd']:  #TODO: instead of all numerical features, perhaps only for price_usd & compX_rate_percent_diff - maaaybe srch_booking_window, orig_destination_distance. Investigate this based on further EDA (boxplots for example)
    # Print initial statistics
    initial_stats = df[feature].describe()
    print(f'Initial stats for {feature}:')
    print(initial_stats)

    # Calculating IQR
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR  # increase to 3 if we want to be less strict
    upper_bound = Q3 + 1.5 * IQR
    
    feature_outliers = ~df[feature].between(lower_bound, upper_bound)
    outliers |= feature_outliers
    
df = df[~outliers]

    # # Filtering out outliers
    # df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]
    
    # # Print updated statistics
    # updated_stats = df[feature].describe()
    # print(f'Updated stats for {feature} after removing outliers:')
    # print(updated_stats)
    # print('\n---\n')

Initial stats for price_usd:
count    4.958347e+06
mean     2.542096e+02
std      1.600124e+04
min      0.000000e+00
25%      8.500000e+01
50%      1.220000e+02
75%      1.849600e+02
max      1.972633e+07
Name: price_usd, dtype: float64


In [7]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,False,NaN,False
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,False,NaN,False
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,False,NaN,False
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,False,NaN,False
5,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,30184,4,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,7.0,False,NaN,False


In [8]:
print(f'Number of rows post outlier removal: {len(df)}')

Number of rows post outlier removal: 4663867


### Imputing Missing Values for Numerical Features

Zero was chosen as the imputation value for the numerical features because it intuitively denotes a missing record, for the competitor data where we have either +1, 0, -1, assuming 0 in the case of missing data assumes no advantage for Expedia or a competitor.

In [9]:
print('Missing values replaced with 0:')
print('------------------------------')
for feature in numerical_features:
    print(f'{feature} - {df[feature].isna().sum()}')
    df.fillna({feature: 0}, inplace=True)

Missing values replaced with 0:
------------------------------
visitor_hist_starrating - 4427242
visitor_hist_adr_usd - 4426170
prop_starrating - 0
prop_review_score - 7089
prop_location_score1 - 0
prop_location_score2 - 1041778
prop_log_historical_price - 0
position - 0
price_usd - 0
srch_length_of_stay - 0
srch_booking_window - 0
srch_adults_count - 0
srch_children_count - 0
srch_room_count - 0
srch_query_affinity_score - 4361229
orig_destination_distance - 1493816
comp1_rate - 4565828
comp1_inv - 4557711
comp1_rate_percent_diff - 4587105
comp2_rate - 2744242
comp2_inv - 2648694
comp2_rate_percent_diff - 4140251
comp3_rate - 3221074
comp3_inv - 3112390
comp3_rate_percent_diff - 4224784
comp4_rate - 4389173
comp4_inv - 4359402
comp4_rate_percent_diff - 4545881
comp5_rate - 2581860
comp5_inv - 2457982
comp5_rate_percent_diff - 3873412
comp6_rate - 4445385
comp6_inv - 4427819
comp6_rate_percent_diff - 4576808
comp7_rate - 4373129
comp7_inv - 4338168
comp7_rate_percent_diff - 4537063
com

### Computing EXP Features for Categorical Features

EXP features involve calculating the probability of a target variable (booking & click in our case) given a certain categorical feature. This is done by grouping the data by the categorical feature and calculating the mean of the booking/click_bool column. This is then merged back into the original dataframe to create the new feature that is in numerical space.

In [10]:
for feature in categorical_features:
    # Initialize new columns for EXP features
    df[f'{feature}_exp_click'] = np.nan
    df[f'{feature}_exp_booking'] = np.nan
    
    # Iterate over each category in the feature
    for category in df[feature].unique():
        # Create mask for the current category
        category_mask = df[feature] == category
        
        # Calculate sums and counts excluding the current row
        # We use sum and count to calculate the mean without the current row
        sum_click = df.loc[category_mask, 'click_bool'].sum()
        count_click = category_mask.sum()
        sum_booking = df.loc[category_mask, 'booking_bool'].sum()
        count_booking = category_mask.sum()
        
        # Assign the leave-one-out mean to the rows of the current category
        # For each row, we use (sum - current value) / (count - 1) to get the leave-one-out mean
        # Important to leave out the current row when calculating the mean to prevent data leakage
        df.loc[category_mask, f'{feature}_exp_click'] = (sum_click - df['click_bool']) / (count_click - 1)
        df.loc[category_mask, f'{feature}_exp_booking'] = (sum_booking - df['booking_bool']) / (count_booking - 1)

# Handle cases where the category only has one row and thus (count - 1) would be zero
for feature in categorical_features:
    df.fillna({f'{feature}_exp_click': df['click_bool'].mean()}, inplace=True)
    df.fillna({f'{feature}_exp_booking': df['booking_bool'].mean()}, inplace=True)

In [11]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,site_id_exp_click,site_id_exp_booking,visitor_location_country_id_exp_click,visitor_location_country_id_exp_booking,prop_country_id_exp_click,prop_country_id_exp_booking
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.0,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.0,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.0,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998
5,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,30184,4,4.5,...,7.0,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998


### Feature Engineering

#### 'Proxy' Prior for Property Position in Results

In [12]:
filtered_df = df[df['random_bool'] == False]  #? Exclude records where the results order is random
mean_positions = filtered_df.groupby('prop_id')['position'].mean().rename('mean_position')
df = df.join(mean_positions, on='prop_id')

In [13]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,click_bool,gross_bookings_usd,booking_bool,site_id_exp_click,site_id_exp_booking,visitor_location_country_id_exp_click,visitor_location_country_id_exp_booking,prop_country_id_exp_click,prop_country_id_exp_booking,mean_position
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,25.861284
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,23.498925
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,24.150000
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,12.739292
5,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,30184,4,4.5,...,False,0.0,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,10.886756


#### Prior of Booking/Clicks per Property

In [14]:
def compute_prior(df, group_field, value_field):
    # Sum and count values per group
    sums = df.groupby(group_field)[value_field].transform('sum')
    count = df.groupby(group_field)[value_field].transform('count')
    
    # Calculate leave-one-out prior
    prior = (sums - df[value_field]) / (count - 1)
    return prior

In [15]:
# Apply function for click and booking bool
df['click_prior'] = compute_prior(df, 'prop_id', 'click_bool')
df['booking_prior'] = compute_prior(df, 'prop_id', 'booking_bool')

# Handling cases with only one record per group
# df['click_prior'].fillna(df['click_bool'].mean(), inplace=True)
# df['booking_prior'].fillna(df['booking_bool'].mean(), inplace=True)
df.fillna({'click_prior': df['click_bool'].mean()}, inplace=True)
df.fillna({'booking_prior': df['booking_bool'].mean()}, inplace=True)

In [16]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,booking_bool,site_id_exp_click,site_id_exp_booking,visitor_location_country_id_exp_click,visitor_location_country_id_exp_booking,prop_country_id_exp_click,prop_country_id_exp_booking,mean_position,click_prior,booking_prior
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,25.861284,0.026187,0.016367
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,23.498925,0.025818,0.015491
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,24.150000,0.009107,0.003643
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,12.739292,0.051282,0.036199
5,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,30184,4,4.5,...,False,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,10.886756,0.045242,0.021841


#### Number of Previous Search Resutls Containing the Property

In [17]:
# Number of occurences "minus the current row"
df['previous_searches'] = df.groupby('prop_id')['prop_id'].transform('count') - 1

In [18]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,site_id_exp_click,site_id_exp_booking,visitor_location_country_id_exp_click,visitor_location_country_id_exp_booking,prop_country_id_exp_click,prop_country_id_exp_booking,mean_position,click_prior,booking_prior,previous_searches
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,25.861284,0.026187,0.016367,611
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,23.498925,0.025818,0.015491,581
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,24.150000,0.009107,0.003643,549
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,12.739292,0.051282,0.036199,663
5,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,30184,4,4.5,...,0.050128,0.025717,0.050624,0.024945,0.044218,0.028998,10.886756,0.045242,0.021841,641


#### Statistical Features for Each Property

Mean, median & std for selected features across all instances of each property

In [19]:
# print all the columns in our df
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [20]:
features_to_stat = [feature for feature in df.columns if df[feature].dtype == 'float64']  #TODO: Decide for which features we should compute statistical features
for feature in features_to_stat:
    print(feature)
    stats = df.groupby('prop_id')[feature].agg(['min', 'max', 'mean', 'median', 'std']).rename(
        columns={'min': f'{feature}_min', 'max': f'{feature}_max', 'mean': f'{feature}_mean', 'median': f'{feature}_median', 'std': f'{feature}_std'})
    df = df.join(stats, on='prop_id')

visitor_hist_starrating
visitor_hist_adr_usd
prop_review_score
prop_location_score1


KeyboardInterrupt: 

In [ ]:
df.head()

### Dropping Features with Many Missing Values
Not done anymore!

In [ ]:
# # create a list of columns to drop where more than 50% of the data is missing
# columns_to_drop = df.columns[df.isnull().mean() > 0.5].tolist()
# pprint(f'Columns that are dropped due to excessive missing values: {columns_to_drop}')
# df.drop(columns=columns_to_drop, inplace=True)

### Normalising/Standardising Numerical Features

Normalisation (scaling data between 0 and 1) or standardisation (shifting the distribution to have a mean of zero and a standard deviation of one) can be beneficial for algorithms that are sensitive to the scale of input data (like SVM or KNN).

In [ ]:
# Example
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Example: Standardizing 'price_usd'
scaler = StandardScaler()
df['price_usd'] = scaler.fit_transform(df[['price_usd']])

# Or for normalization
# scaler = MinMaxScaler()
# df['price_usd'] = scaler.fit_transform(df[['price_usd']])

### Encoding Categorical Variables

In [ ]:
# Example for 1-hot encoding a categorical feature
df = pd.get_dummies(df, columns=['prop_country_id'], drop_first=True)

### Feature Engineering

Ideas for this:
1. Time Features: Break down the date_time column into year, month, day, weekday, and hour components. Time could affect booking patterns.
2. Interaction Features: Create features that represent interactions between the customer’s historical preferences and property attributes, like the difference between the user’s average star rating and the property’s star rating.
3. Textual and Categorical Embeddings: If there are textual descriptions available or high-cardinality categorical variables, consider using embeddings or hashing techniques to reduce their dimensionality.

In [ ]:
# Create time-based features
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['weekday'] = df['date_time'].dt.weekday
df['hour'] = df['date_time'].dt.hour

# Interaction feature: difference between user's average star rating and property's star rating
df['starrating_diff'] = df['visitor_hist_starrating'] - df['prop_starrating']

# Re-check the dataset
df.head()